In [127]:
import torch
import numpy as np  
from torch import nn, tensor, Tensor
from torch.nn import functional


In [133]:
gt_length = 20
#rel_x = np.random.randint(0,10,20)
rel_x = np.array(range(0,gt_length))
#rel_y = np.random.randint(0,10,20)
rel_y = np.array(range(0,gt_length))
start = np.random.randint(0,2,gt_length)

gt = np.c_[rel_x, rel_y, start][np.newaxis, np.newaxis] # BATCH, CHANNEL, WIDTH, HEIGHT
pred = gt.copy()
pred[:,:,7,0] = 8

# Batch it up
# gt = np.r_[gt, pred]
# print(gt.shape)

In [154]:
#m = nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
KERNEL_LENGTH = 9
CHANNELS = 1
KERNEL = Tensor(np.cumsum([.1]*KERNEL_LENGTH)[np.newaxis].transpose(1,0)).repeat(1, CHANNELS, 1, 1)
#KERNEL = Tensor(np.cumsum([.1]*KERNEL_LENGTH)).repeat(1, CHANNELS, 1, 1)
INVERSE_KERNEL = torch.flip(KERNEL, dims=(2,))

def conv_weight(gt, pred):
    """ BATCH, CHANNEL, HEIGHT (VOCAB SIZE), WIDTH (GT LENGTH)
        gt.shape[-1]
    """
    # nn.module way
    #         conv = nn.Conv2d(1, 1, kernel_length, stride=1, padding=[0,kernel_length-1], dilation=1, groups=1, bias=False, padding_mode='zeros')
    #         conv.weight = nn.Parameter(Tensor(kernel_legnth).repeat(1, channels, 1, 1))
    #         print(conv.weight)
    #         out = conv(Tensor(gt[np.newaxis]))

    # Numpy convolution - sum window
    # print(np.convolve(gt[0,0,0],np.ones(kernel_length,dtype=int),'valid'))
    width = gt.shape[2]
    # Functionary way
    conv_gt = functional.conv2d(Tensor(gt), INVERSE_KERNEL, padding=[KERNEL_LENGTH - 1,0])
    conv = functional.conv2d(Tensor(pred), KERNEL, padding=[KERNEL_LENGTH - 1,0])
    cumsum = np.zeros(60).reshape(1, 1, width, 3)
    cumsum[:, :, KERNEL_LENGTH:, :] = np.cumsum(gt, axis=2)[:, :, :width - KERNEL_LENGTH, :]
    return conv[:, :, :width, :] + conv_gt[:, :, :width, :] + Tensor(cumsum)


# Anything with itself should be equivalent to a cumulative sum
x = conv_weight(gt,gt)
np.testing.assert_almost_equal(x, np.cumsum(gt,axis=2), decimal=5)

# Should get back on track
x = conv_weight(gt,pred)
np.testing.assert_almost_equal(x[:,:,-4:], np.cumsum(gt,axis=2)[:,:,-4], decimal=5)
np.testing.assert_almost_equal(x[:,:,0:6], np.cumsum(gt,axis=2)[:,:,0:6], decimal=5)

# With batching
gt2 = np.r_[gt,gt]
x = conv_weight(gt2,gt2)
np.testing.assert_almost_equal(x, np.cumsum(gt,axis=2), decimal=5)


AssertionError: 
Arrays are not almost equal to 5 decimals

(shapes (1, 1, 4, 3), (1, 1, 3) mismatch)
 x: array([[[[136., 136.,   8.],
         [153., 153.,   8.],
         [171., 171.,   9.],
         [190., 190.,   9.]]]], dtype=float32)
 y: array([[[136, 136,   8]]])

In [151]:
print(INVERSE_KERNEL, KERNEL)
print(KERNEL.shape)

tensor([[[[0.9000],
          [0.8000],
          [0.7000],
          [0.6000],
          [0.5000],
          [0.4000],
          [0.3000],
          [0.2000],
          [0.1000]]]]) tensor([[[[0.1000],
          [0.2000],
          [0.3000],
          [0.4000],
          [0.5000],
          [0.6000],
          [0.7000],
          [0.8000],
          [0.9000]]]])
torch.Size([1, 1, 9, 1])


In [82]:
cumsum[:,:,:,kernel_length:]

array([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]])

20